# Fruit Toy Problem E

This problem illustrates the use of pricing data and storage in the openLCA database.

## Problem Statement
 
A citrus fruit producer wishes to purchase fruit over a 2 day period in order to minimise cost. The fruit supply options are globally sourced lemon, mandarins or oranges. The producer can choose when to purchase the fruit, but we assume the environmental impact of production and the transport of fruit to a processing plant is the same for each fruit and independent of time.

Over a time horizon of 2 days, the producer is required to deliver 2 kg of fruit. The cost of producing 1kg of fruit is shown in the table below.

| Fruit | Day 1 | Day 2 |
| --- | --- | --- |
| Lemon | £1 | £4 |
| Orange | £2 | £3 |
| Mandarin | £3 | £2 |

The cost of storing 1kg of fruit at the processing plant is £1 per day.

## Simple specification

In this problem, we minimise the cost of fruit production over a two day period $t\in T=\{1,2\}$. We ignore the environmental impact of production and transport.

The objective is

$$
\min_{f_m, p_m, t}\sum_{f_m, p_m, t} Flow_{f_m, p_m, t}\phi_{f_m, p_m, t} + \sum_{f_s, p_s, t} S_{f_s, p_s, t}\phi_{f_s, p_s, t}.
$$

where the cost of producing 1kg of fruit in a day is given by $\phi_{f_m, p_m, t}$ in the table above and the cost of storing each kg of fruit is $\phi_{f_s, p_s, t}=1$.

There is no initial stored fruit.

There total demand is $D^{total}=2kg$ of fruit so

$$
\sum_{f_m, p_m, t} Flow_{f_m, p_m, t} \geq D^{total} = 2.
$$

All the material flow of fruit at time $1$, must be stored over time period 2 so we require 

$$
S_{f_{s_i}, p_{s_i}, 2} = Flow_{f_{m_i}, p_{m_i}, 1}
$$

We also require $Flow_{f_m, p_m, t} \geq 0$ and $S_{f_s, p_s, t}\geq 0$.

## Pyomo implementation

In [1]:
from pyomo.environ import * 
import pandas as pd

### Create a concrete model

In [2]:
model = ConcreteModel()

### Define sets. Will come from mola, but currently hardcoded.

Flows have units kg per day. Time has units of day.

In [3]:
model.t = Set(initialize=[1,2], doc='Time period')
model.AF = Set(initialize=['Lemon', 'Orange', 'Mandarin'], doc='All flows')
model.Fm = Set(initialize=['Lemon', 'Orange', 'Mandarin'], doc='Material flows to optimise')
model.Fs = Set(initialize=['Lemon Storage', 'Orange Storage', 'Mandarin Storage'], doc='Service flows to optimise')

### Define parameters.

#### Get cost parameters. Will come from mola, but currently hardcoded.

In [4]:
m = {}
for i, f in enumerate(model.Fm):
    m[f, 1] = i + 1
    m[f, 2] = 4 - i
model.phimt = Param(model.Fm, model.t, initialize=m, within=Any, doc='Cost of material flow fm at time t')
model.phist = Param(model.Fs, model.t, initialize=1, within=Any, doc='Cost of material flow fm at time t')
model.phimt.pprint()

phimt : Cost of material flow fm at time t
    Size=6, Index=phimt_index, Domain=Any, Default=None, Mutable=False
    Key             : Value
       ('Lemon', 1) :     1
       ('Lemon', 2) :     4
    ('Mandarin', 1) :     3
    ('Mandarin', 2) :     2
      ('Orange', 1) :     2
      ('Orange', 2) :     3


### Define continuous decision variables

In [5]:
model.y = Var(model.Fm | model.Fs, model.t, within=NonNegativeReals, doc='Decision variables')
model.y.pprint()

y : Decision variables
    Size=12, Index=y_index
    Key                     : Lower : Value : Upper : Fixed : Stale : Domain
               ('Lemon', 1) :     0 :  None :  None : False :  True : NonNegativeReals
               ('Lemon', 2) :     0 :  None :  None : False :  True : NonNegativeReals
       ('Lemon Storage', 1) :     0 :  None :  None : False :  True : NonNegativeReals
       ('Lemon Storage', 2) :     0 :  None :  None : False :  True : NonNegativeReals
            ('Mandarin', 1) :     0 :  None :  None : False :  True : NonNegativeReals
            ('Mandarin', 2) :     0 :  None :  None : False :  True : NonNegativeReals
    ('Mandarin Storage', 1) :     0 :  None :  None : False :  True : NonNegativeReals
    ('Mandarin Storage', 2) :     0 :  None :  None : False :  True : NonNegativeReals
              ('Orange', 1) :     0 :  None :  None : False :  True : NonNegativeReals
              ('Orange', 2) :     0 :  None :  None : False :  True : NonNegativeReals
   

### Define constraints

In [6]:
def demand_constraint(model):
    s = sum(model.y[fm,t] for fm in model.Fm for t in model.t)
    return s >= 2
model.demand_constraint = Constraint(rule=demand_constraint)

model.storage_constraint_list = ConstraintList()
model.storage_constraint_list.add(model.y['Lemon',1] == model.y['Lemon Storage',2])
model.storage_constraint_list.add(model.y['Orange',1] == model.y['Orange Storage',2])
model.storage_constraint_list.add(model.y['Mandarin',1] == model.y['Mandarin Storage',2])
model.storage_constraint_list.pprint()

storage_constraint_list : Size=3, Index=storage_constraint_list_index, Active=True
    Key : Lower : Body                                  : Upper : Active
      1 :   0.0 :       y[Lemon,1] - y[Lemon Storage,2] :   0.0 :   True
      2 :   0.0 :     y[Orange,1] - y[Orange Storage,2] :   0.0 :   True
      3 :   0.0 : y[Mandarin,1] - y[Mandarin Storage,2] :   0.0 :   True


### Define objective

In [7]:
def objective_rule(model):
    obj_material = sum(model.y[f,t]*model.phimt[f,t] for f in model.Fm for t in model.t)
    obj_service = sum(model.y[f,t]*model.phist[f,t] for f in model.Fs for t in model.t)
    return obj_material+obj_service
model.obj = Objective(rule=objective_rule, sense=minimize)
model.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : y[Lemon,1] + 4*y[Lemon,2] + 2*y[Orange,1] + 3*y[Orange,2] + 3*y[Mandarin,1] + 2*y[Mandarin,2] + y[Lemon Storage,1] + y[Lemon Storage,2] + y[Orange Storage,1] + y[Orange Storage,2] + y[Mandarin Storage,1] + y[Mandarin Storage,2]


### Apply solver

In [8]:
opt = SolverFactory("glpk")
results = opt.solve(model)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 4.0
  Upper bound: 4.0
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 13
  Number of nonzeros: 13
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.019842863082885742
# ----------------------------------------------------------
#   Solution Information
# --------------------------------------

### Process results

In [9]:
model.y.pprint()

y : Decision variables
    Size=12, Index=y_index
    Key                     : Lower : Value : Upper : Fixed : Stale : Domain
               ('Lemon', 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
               ('Lemon', 2) :     0 :   0.0 :  None : False : False : NonNegativeReals
       ('Lemon Storage', 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
       ('Lemon Storage', 2) :     0 :  -0.0 :  None : False : False : NonNegativeReals
            ('Mandarin', 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
            ('Mandarin', 2) :     0 :   2.0 :  None : False : False : NonNegativeReals
    ('Mandarin Storage', 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
    ('Mandarin Storage', 2) :     0 :  -0.0 :  None : False : False : NonNegativeReals
              ('Orange', 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
              ('Orange', 2) :     0 :   0.0 :  None : False : False : NonNegativeReals
   

In [10]:
model.obj.pprint()
model.obj.value()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : y[Lemon,1] + 4*y[Lemon,2] + 2*y[Orange,1] + 3*y[Orange,2] + 3*y[Mandarin,1] + 2*y[Mandarin,2] + y[Lemon Storage,1] + y[Lemon Storage,2] + y[Orange Storage,1] + y[Orange Storage,2] + y[Mandarin Storage,1] + y[Mandarin Storage,2]
    deprecated. Use the .expr property getter instead


4.0

# Using Mola Specification

Next we try and rewrite this model using the built-in mola model specification v5. In the specification the service flow is related to the material storage flow by

$$
S_{f_s,p_s,t} = \sum_{f_m, p_m} L_{f_m, p_m, f_s, p_s}S_{f_m, p_m, t},
$$

The service flow $S_{f_s, p_s, t}$ refers to the service process $p_s$ because in openLCA the process holds attributes like location rather than the flow. In this problem, there is only one material storage flow $S_{f_m, p_m, t}$ for each country and so there is a direct relationship:

$$
L_{f_m, p_m, f_s, p_s} = 1
$$

if the storage process $p_s$ is in the same country as $p_m$.

# Using openLCA data

We aim to obtain the price of fruit from the openLCA database and use this in the optimisation problem. It not clear that we have relevant storage data, so we may have to invent this. Fruit prices are also likely to be static.

In [11]:
import qgrid
#import importlib
import mola.dataview as dv
import mola.dataimport as di
from importlib import reload
reload(dv)

dbconn = di.get_sqlite_connection()

In [12]:
#dfr = dv.get_table(dbconn, 'TBL_EXCHANGES')
#dfr